# Data Wrangling for Student Performance Datasets

This notebook demonstrates data wrangling for the student performance datasets (`mat.arff`, `por.arff`, and `dataset.csv`). The processed data will be saved in the `processed_data` folder.

### Dependancies and frameworks
Load the two required dependencies:

- [Pandas](https://pandas.pydata.org/) is library that allows us to handle data for wrangling and visualisation.
- [sklearn](https://scikit-learn.org/stable/) A framework for training Machine Learning, we will use this for wrangling, but also applies to training and testing.
- [os, IO](https://docs.python.org/3/library) Default packages installed with python, allows us to create, save and edit files with basic string functions.

In [31]:
# Import frameworks
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import os
from io import StringIO


#### Load the datasets

In [32]:
# Load ARFF files as CSV
def load_arff_as_csv(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    data_start = False
    data = []
    for line in lines:
        if data_start:
            data.append(line.strip())
        if line.strip().lower() == '@data':
            data_start = True
    return pd.read_csv(StringIO('\n'.join(data)), header=None)

# Load ARFF files
mat_df = load_arff_as_csv('data/mat.arff')
por_df = load_arff_as_csv('data/por.arff')

# Set column names for mat.arff
mat_columns = ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3']
mat_df.columns = mat_columns

# Set column names for por.arff
por_columns = ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2', 'G3']
por_df.columns = por_columns

# Load CSV file (which is actually in ARFF format)
csv_columns = ['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G3']

# Use the same ARFF loading function for dataset.csv
csv_df = load_arff_as_csv('data/dataset.csv')
csv_df.columns = csv_columns

#### Dealing with null values

In [33]:
# Remove Null values
def remove_nulls(df):
    df = df.dropna()
    return df

mat_df = remove_nulls(mat_df)
por_df = remove_nulls(por_df)
csv_df = remove_nulls(csv_df)

#### Remove Duplicates

In [34]:
# Remove duplicates
def remove_duplicates(df):
    df = df.drop_duplicates()
    return df

mat_df = remove_duplicates(mat_df)
por_df = remove_duplicates(por_df)
csv_df = remove_duplicates(csv_df)

#### Replace data

In [35]:
# Replace data
def replace_data(df, column):
    df[column] = df[column].apply(lambda x: x.lower())
    return df

mat_df = replace_data(mat_df, 'sex')
por_df = replace_data(por_df, 'sex')
csv_df = replace_data(csv_df, 'sex')

#### Remove outliers

In [36]:
# Remove outliers
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    df = df[(df[column] >= Q1 - 1.5 * IQR) & (df[column] <= Q3 + 1.5 * IQR)]
    return df

mat_df = remove_outliers(mat_df, 'age')
por_df = remove_outliers(por_df, 'age')
csv_df = remove_outliers(csv_df, 'age')

#### Scaling features to a common range

In [37]:
# Scale features
# this would normally be something like this
# scaler = MinMaxScaler()
# mat_df[['age', 'absences', 'G3']] = scaler.fit_transform(mat_df[['age', 'absences', 'G3']])
# por_df[['age', 'absences', 'G3']] = scaler.fit_transform(por_df[['age', 'absences', 'G3']])
# csv_df[['age', 'absences', 'G3']] = scaler.fit_transform(csv_df[['age', 'absences', 'G3']])

# but for this use case, this data should not be scaled

#### Save the wrangled data

In [42]:
# Create directory if it doesn't exist
os.makedirs('processed_data', exist_ok=True)

# Save the processed data files
mat_df.to_csv('processed_data/Pmat.csv', index=False)
por_df.to_csv('processed_data/Ppor.csv', index=False)
csv_df.to_csv('processed_data/Pdataset.csv', index=False)
# Raw processed data

#### Split the data into training and testing sets

In [43]:
# Split the data
X = csv_df.drop('G3', axis=1)  # Features
y = csv_df['G3']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Debugging statements
print(f"Number of rows in csv_df: {len(csv_df)}")
print(f"Number of rows in X_train: {len(X_train)}")
print(f"Number of rows in X_test: {len(X_test)}")

# Save all processed datasets
mat_df.to_csv('processed_data/Pmat.csv', index=False)
por_df.to_csv('processed_data/Ppor.csv', index=False)
csv_df.to_csv('processed_data/Pdataset.csv', index=False)

# Save the split data
X_train.to_csv('processed_data/X_train.csv', index=False)
X_test.to_csv('processed_data/X_test.csv', index=False)
y_train.to_csv('processed_data/y_train.csv', index=False)
y_test.to_csv('processed_data/y_test.csv', index=False)

Number of rows in csv_df: 648
Number of rows in X_train: 518
Number of rows in X_test: 130
